In [2]:
from qutip import *
import numpy as np
from scipy import integrate
from helper_code_qutip import * 
import scipy.io

In [54]:
##FOr referencing F_lists and other_stuff
#matlab_data1 = scipy.io.loadmat('../matlab/thermal_data1.mat',mat_dtype=False)  # is set to True, givex complex casting to real errors..
#matlab_data2 = scipy.io.loadmat('../matlab/thermal_data3.mat',mat_dtype=False)  # is set to True, givex complex casting to real errors..

#redfield_mat_data = scipy.io.loadmat(f'../matlab/ness_data_new_NL1=2,NL2=2,NM=2,e=0.00,beta_r=0.5,beta_l=1.0,g=0.0100,s=0.75.mat',mat_dtype=False)
#L2_red = np.squeeze(redfield_mat_data["L2_red"])
#print(L2_red.shape)

print(sigmay())


Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=CSR, isherm=True
Qobj data =
[[0.+0.j 0.-1.j]
 [0.+1.j 0.+0.j]]


In [32]:
NL1 = 2
NL = NL1
NR = NL1
NM = 2
N = NL + NR + NM
dL = 2**NL
dR = 2**NR
dM = 2**NM
d = dL * dR * dM
dims = [2]*N

create_sm_list_left = [create_sm(N,i + 1) for i in range(NL)]
create_sm_list_right = [create_sm(N,NM + NL + i + 1) for i in range(NR)]

epsilon = 0.01 #system bath coupling strength

In [41]:
def integral1(i,k,s,tb,beta,mu,gamma,eigenenergies,limit_value = 700,b=50):
    freq=eigenenergies[k]-eigenenergies[i]
    if( np.absolute(freq) >= 1/10**10):
        integral = (-1.0j/(2*np.pi))*integrate.quad(func1/freq,0,b,args=(s,tb,beta,mu,gamma),limit=limit_value,weight='cauchy',wvar=eigenenergies[k]-eigenenergies[i])[0]
    else:
        integral = (-1.0j/(2*np.pi))*integrate.quad(func2,0,b,args=(s,tb,beta,mu,gamma),limit=limit_value)[0]
    return integral

def integral2(i,k,s,tb,gamma,eigenenergies,limit_value = 700,b=50):
    freq=eigenenergies[k]-eigenenergies[i]
    if( np.absolute(freq) >= 1/10**10):
        integral = (-1.0j/(2*np.pi))*integrate.quad(spectral_bath,0,b,args=(s,tb,gamma),limit=limit_value,weight='cauchy',wvar=eigenenergies[k]-eigenenergies[i])[0]
    else:
        integral = (-1.0j/(2*np.pi))*integrate.quad(spectral_bath_2,0,b,args=(s,tb,gamma),limit=limit_value)[0]
    return integral

def C(i,k,s,tb,beta,mu,gamma,eigenenergies):
    val = integral1(i,k,s,tb,beta,mu,gamma,eigenenergies) + 0.5*(func1(eigenenergies[k]-eigenenergies[i],s,tb,beta,mu,gamma))

    return val

def D(i,k,s,tb,beta,mu,gamma,eigenenergies):
    val = integral1(i,k,s,tb,beta,mu,gamma,eigenenergies) + integral2(i,k,s,tb,gamma,eigenenergies) + 0.5*(spectral_bath(eigenenergies[k]-eigenenergies[i],s,tb,gamma)+func1(eigenenergies[k]-eigenenergies[i],s,tb,beta,mu,gamma))
    return val

def L2_red(eigstates,number, constant11,constant12,constant21,constant22,dims):
    data = []
    N = np.prod(dims)
    basis_ops = [basis(N, i) * basis(N, j).dag() for i in range(N) for j in range(N)]
    for op in basis_ops:
        op.dims = [dims,dims]
        sum_op = 0 * op
        for i in range(number):
            for k in range(number):
                vi = eigstates[i]
                vk = eigstates[k]

                proj_i = vi * vi.dag()
                proj_k = vk * vk.dag()

                proj_i.dims = [dims, dims]
                proj_k.dims = [dims, dims]

                for l in range(len(create_sm_list_left)):
                    create_sm_list_left[l].dims = [dims, dims]
                    op1 = commutator(op * proj_i * create_sm_list_left[l] * proj_k, create_sm_list_left[l].dag()) * constant11[i, k]
                    op2 = commutator(create_sm_list_left[l].dag(), proj_i * create_sm_list_left[l] * proj_k * op) * constant12[i, k]
                    sum_op += op1 + op1.dag() + op2 + op2.dag()

                for l in range(len(create_sm_list_right)):
                    create_sm_list_right[l].dims = [dims, dims]
                    op1 = commutator(op * proj_i * create_sm_list_right[l] * proj_k, create_sm_list_right[l].dag()) * constant21[i, k]
                    op2 = commutator(create_sm_list_right[l].dag(), proj_i * create_sm_list_right[l] * proj_k * op) * constant22[i, k]
                    sum_op += op1 + op1.dag() + op2 + op2.dag()

        data.append(operator_to_vector(sum_op))

    # Stack the results into a superoperator matrix
    superop_matrix = np.hstack([col.full() for col in data]).T
    L2_superop = Qobj(superop_matrix, dims=[dims,dims], type='super')

    return L2_superop

def optimized_L2_red(eigstates, C1, C2, D1, D2, create_sm_list_left, create_sm_list_right, dims):
    """
    Optimized function to construct the L2_red superoperator, corrected for summation over all operators.
    
    Args:
        eigstates (list): List of eigenstate Qobj vectors.
        C1, C2, D1, D2 (np.ndarray): Matrices of constants.
        create_sm_list_left (list): List of left-side creation/destruction operators.
        create_sm_list_right (list): List of right-side creation/destruction operators.
        dims (list): Dimensions of the system.

    Returns:
        Qobj: The L2_red superoperator.
    """
    superop_total = 0
    N = len(eigstates)

    # Combine all operator lists for easier looping
    all_op_lists = [(create_sm_list_left, C1, C2), (create_sm_list_right, D1, D2)]

    # Loop through all eigenstates for the alpha and gamma indices
    for alpha in range(N):
        for gamma in range(N):
            E_alpha = eigstates[alpha] * eigstates[alpha].dag()
            E_gamma = eigstates[gamma] * eigstates[gamma].dag()
            
            # Loop through both the left and right operator sets
            for op_list, C_matrix, D_matrix in all_op_lists:
                for op in op_list:
                    # Terms corresponding to C and C*
                    term1 = spre(E_alpha * op * E_gamma) * spost(op.dag()) * C_matrix[alpha, gamma]
                    term2 = spre(op.dag()) * spost(E_alpha * op * E_gamma) * D_matrix[alpha, gamma]
                    
                    # Add terms and their Hermitian conjugates
                    superop_total += term1 + term1.dag() + term2 + term2.dag()

    # Re-normalize dimensions and return
    superop_total.dims = [[dims, dims], [dims, dims]]
    return superop_total

In [46]:

##We will utilise the existing code to get the steady state of the CVX system, and the Redfield system, and then subtract the off diagonal terms, mod it, and add them up to get the final tau_coh measure

##left and right betas fixed, changing g. Data should be from coh_data files...

#NL1 = 2
e = 0.00
beta_r = 1.0
beta_l = 1.0
g = 0.0100

matlab_data_g = scipy.io.loadmat(f'../matlab/data_plotting_vss/coh_data_new2_NL1={NL1},e={e:.2f},beta_r={beta_r:.1f},g={g:.4f},ham_type=1.mat',mat_dtype=False)

##we extract the s_list, gamma_lists and ls_lists

s_list = matlab_data_g["s_list"][0]
s_list = s_list.astype('float')
print('slist is ', s_list)

gamma_left_list = matlab_data_g["gamma_left"][0]
#print('gamma_left_list is ', gamma_left_list)
gamma_right_list = matlab_data_g["gamma_right"][0]
ls_left_list = matlab_data_g["ls_left"][0]
ls_right_list = matlab_data_g["ls_right"][0]

gamma_left_arr = []
gamma_right_arr = []
ls_left_arr = []
ls_right_arr = []

for i in range(len(s_list)):    
    gamma_left_arr.append(np.squeeze(gamma_left_list[i]))
    gamma_right_arr.append(np.squeeze(gamma_right_list[i]))
    ls_left_arr.append(np.squeeze(ls_left_list[i]))
    ls_right_arr.append(np.squeeze(ls_right_list[i]))

print('gamma_left_arr example is ', gamma_left_arr[0].shape)



slist is  [0.75 1.   1.25 1.5  1.75 2.   2.25 2.5  2.75 3.  ]
gamma_left_arr example is  (15, 15)


In [ ]:
##Save the files according to s:
for i in range(len(s_list)):
    s = s_list[i]
    np.savetxt(f"gamma_left_betar={beta_r:.1f},g={g:.4f},s={s:.2f}",gamma_left_arr[i])
    np.savetxt(f"gamma_right_betar={beta_r:.1f},g={g:.4f},s={s:.2f}",gamma_right_arr[i])
    np.savetxt(f"ls_left_betar={beta_r:.1f},g={g:.4f},s={s:.2f}",ls_left_arr[i])
    np.savetxt(f"ls_right_betar={beta_r:.1f},g={g:.4f},s={s:.2f}",ls_right_arr[i])

In [ ]:
## Now, we have to form the steady states for the CVX states
NL = NL1
NR = NL1
NM = 2
N = NL + NR + NM
dL = 2**NL
dR = 2**NR
dM = 2**NM
d = dL * dR * dM
dims = [2]*N

create_sm_list_left = [create_sm(N,i + 1) for i in range(NL)]
create_sm_list_right = [create_sm(N,NM + NL + i + 1) for i in range(NR)]

w0list = matlab_data_g["w0list"][:,0]
w0list = w0list.astype('float')
print('w0list is ', w0list)

glist = matlab_data_g["glist"][:,0]
glist = glist.astype('float')
print('glist is ', glist)

delta = 1



H_S = create_hamiltonian(w0list,glist,delta,N)
eigenenergies, eigenstates = H_S.eigenstates()
number = len(eigenenergies) # should be 2^N



w0list is  [2. 2. 2. 2. 2. 2.]
glist is  [-0.01 -0.01 -0.01 -0.01 -0.01]


In [17]:
matlab_F_list1 = matlab_data_g["F1"]
F_list1 = []

matlab_F_list2 = matlab_data_g["F2"]
F_list2 = []


for index in range(2**dL):
    matrix1 = np.asmatrix(matlab_F_list1[0,index])
    matrix2 = np.asmatrix(matlab_F_list2[0,index])

    F_list1.append(Qobj(matrix1))
    F_list2.append(Qobj(matrix2))
    
if not basis_is_orthonormal(F_list1):
    print("WARNING : Basis 1 is NOT orthonormal")

if not basis_is_orthonormal(F_list2):
    print("WARNING : Basis 2 is NOT orthonormal")

F_list1.pop() #removes the last indentity basis ..
print(F_list2[-1]) # should be identity matrix
F_list2.pop()

for index in range(2**dL-1):
    F_list1[index].dims = [dims,dims] # makes sure mutliplication will work and QuTiP wont throw errors. 
    F_list2[index].dims = [dims,dims]



Quantum object: dims=[[64], [64]], shape=(64, 64), type='oper', dtype=Dense, isherm=True
Qobj data =
[[0.125 0.    0.    ... 0.    0.    0.   ]
 [0.    0.125 0.    ... 0.    0.    0.   ]
 [0.    0.    0.125 ... 0.    0.    0.   ]
 ...
 [0.    0.    0.    ... 0.125 0.    0.   ]
 [0.    0.    0.    ... 0.    0.125 0.   ]
 [0.    0.    0.    ... 0.    0.    0.125]]


In [47]:
rho_cvx_list = []

for i in range(len(s_list)):
    print("s = ", s_list[i])
    gamma_matrix1 = gamma_left_arr[i]
    gamma_matrix2 = gamma_right_arr[i]

    [D1,U1] = np.linalg.eigh(gamma_matrix1)
    [D2,U2] = np.linalg.eigh(gamma_matrix2)

    L_list1 = []
    Udagger1 = np.transpose(np.conj(U1))
    L_list2 = []
    Udagger2 = np.transpose(np.conj(U2))

    for j  in range(dL**2-1):
        op1 = 0
        op2 = 0
        for betaindex in range(dL**2-1):    
            op1 = op1+Udagger1[j,betaindex]* F_list1[betaindex]
            op2 = op2+Udagger2[j,betaindex]* F_list2[betaindex]
        L_list1.append(op1)
        L_list2.append(op2)

    tildeL_list1 = []
    tildeL_list2 = []

    for index in range(dL**2-1):
        tildeL_list1.append(epsilon*np.sqrt(D1[index])*L_list1[index])
        tildeL_list2.append(epsilon*np.sqrt(D2[index])*L_list2[index])

    tildeL_list = []
    for i1 in tildeL_list1:
        tildeL_list.append(i1)

    for i2 in tildeL_list2:
        tildeL_list.append(i2)

    H_LS_matrix1 = ls_left_arr[i]
    H_LS_matrix2 = ls_right_arr[i]

    H_LS1 = epsilon**2*tensor(Qobj(H_LS_matrix1),qeye(dM*dR))
    H_LS2 = epsilon**2*tensor(qeye(dL*dM),Qobj(H_LS_matrix2))
    H_LS1.dims = [dims,dims]
    H_LS2.dims = [dims,dims]
    rho_cvx = steadystate(H_S + H_LS1 + H_LS2, tildeL_list)
    rho_cvx_list.append(rho_cvx)

    L_operator = liouvillian(H_S+H_LS1+H_LS2, tildeL_list)

    L_eigen = L_operator.eigenenergies() ##Takes tie, but important to check
    print("Smallest eigenvalues of L are ", L_eigen[-3:])


s =  0.75
Smallest eigenvalues of L are  [-2.94088605e-07+2.07507270e+00j -2.94088603e-07-2.07507270e+00j
  6.42728959e-19-4.55945325e-16j]
s =  1.0
Smallest eigenvalues of L are  [-3.25127013e-07-2.07467609e+00j -3.25127013e-07+2.07467609e+00j
 -5.60377045e-19-1.01999585e-16j]
s =  1.25
Smallest eigenvalues of L are  [-3.26731820e-07-2.07473091e+00j -3.26731819e-07+2.07473091e+00j
 -2.60108649e-19+2.37019100e-16j]
s =  1.5
Smallest eigenvalues of L are  [-3.29922597e-07+2.07483318e+00j -3.29922593e-07-2.07483318e+00j
  9.30378047e-20+1.68918339e-15j]
s =  1.75
Smallest eigenvalues of L are  [-3.23602100e-07+2.00068511e+00j -3.23602098e-07-2.00068511e+00j
  1.28306024e-19+7.86022719e-16j]
s =  2.0
Smallest eigenvalues of L are  [-3.22750389e-07+2.07573526e+00j -3.22750389e-07-2.07573526e+00j
  1.02396376e-19+9.25115860e-16j]
s =  2.25
Smallest eigenvalues of L are  [-2.15549030e-07+2.00183710e+00j -2.15549025e-07-2.00183710e+00j
  1.45829272e-20-1.90115026e-16j]
s =  2.5
Smallest eigen

In [48]:
 ## We have the Redfield steady state calculated from the cluster code, so now we just need to compare the sum of coherences of the mod of the difference of the two steady states, and then add them up to get the final tau_coh measure

tau_coh_list = []

for i in range(len(s_list)):
    print("s = ", s_list[i])
    rho_cvx = rho_cvx_list[i]
    rho_red = np.loadtxt(f"rho_red_s={s_list[i]:.2f}_NL1={NL1}_e={e:.2f}_beta_r={beta_r:.1f}_g={g:.4f}.txt",dtype=complex)
    rho_red = Qobj(rho_red, dims=[dims, dims])
    
    ## Now we calculate the mod of the difference of the two steady states
    diff = rho_cvx - rho_red
    diff_mod = diff.full()
    
    ## Now we sum the off-diagonal terms
    tau_coh = 0
    for j in range(2**N):
        for k in range(2**N):
            if j != k:
                tau_coh += np.abs(diff_mod[j,k])
    print(f"tau_coh for s={s_list[i]:.2f} is {tau_coh:.4f}")
    tau_coh_list.append(tau_coh)

# Save the tau_coh values to a file
np.savetxt(f"tau_coh_s={s_list[0]:.2f}_NL1={NL1}_e={e:.2f}_beta_r={beta_r:.1f}_g={g:.4f}.txt", tau_coh_list)

print("All tau_coh values saved successfully.")
    


    

s =  0.75
tau_coh for s=0.75 is 213.3225
s =  1.0
tau_coh for s=1.00 is 147.6660
s =  1.25
tau_coh for s=1.25 is 253.1882
s =  1.5
tau_coh for s=1.50 is 666.9555
s =  1.75
tau_coh for s=1.75 is 1539.7473
s =  2.0
tau_coh for s=2.00 is 550.4249
s =  2.25
tau_coh for s=2.25 is 242.8148
s =  2.5
tau_coh for s=2.50 is 41.3143
s =  2.75
tau_coh for s=2.75 is 8.0761
s =  3.0
tau_coh for s=3.00 is 11.4775
All tau_coh values saved successfully.
